In [ ]:
# From: https://github.com/vibhor98/Neural-Machine-Translation-using-Keras

In [1]:
import numpy as np
# x = np.ndarray(3, dtype='str')
# x[1] = 'ksd'

In [5]:

def read_data(): 
    src_data  = open("data/giga-fren.release2.fixed.en", "r", errors='ignore')
    src = []
    i = 0
    for line in src_data:
        if i % 1000 == 0:
            src.append(line)
        i+=1

    tgt_data = open("data/giga-fren.release2.fixed.fr", "r",errors='ignore')
    tgt = []
    i = 0
    for line in tgt_data:
        if i % 10000 == 0: 
            tgt.append(line)
        i+=1
        
    return src, tgt


src, tgt = read_data()

print(src)
print(tgt)


['\x1f\x08\x08W\x00\x03giga-fren.release2.fixed.en\x00[n\x1c9wS\x10\x06\x7f\x06h\t$s7\x03,ٖlEXٲ\x1cU\x15{XU\'& /dd$wȺK\tK_X|;.\x16X\x0b6:n>q5\x16\x1fz];9$enU\'^g\x0c3"1*\x0cx1\'Kx6~m͆#I\x0c<??>Vwffn\x14xʘt{֙KSձ.tÚWza&72*\u05ebI$N?yu\u9ff4-\'wK])[.yr4jreccb_PղcM]\x17lҍ?mYZWO:ίG\x17fo\x0b=Z,9F\x04)\x18W[_Bbg\x05Tt\x12WT&_9Vm_\n', '\x0f3uhV ,:5F)\x17\x1c\\\x01\x06ѦQxTEvn[FA\x01/N\x1d`C&\t\x1fB\x1e\x03U\x1b|+[7t\t4\n', 'vK0%딢\x13i<M%\x1d҃\x1d(\'<S"WE&*^\x10\x04st\x10a\x0e#pV\x10/>D\x14I\x1axr\x15i庿VQ|F.e^;4{^A-㗯~%B\x10i\x0c!\x11Ɋ.İdU\x1eWPfzE\x0f6*x\x14\x1dgVu0E\x10\x14N\x16u*u߱\x02:CF),s?4\x17>7ip\x00\x16 ]K䓋_)]\x0f-u6^_<dv},FjM[4\tϟ(d}(= \x18\x03w]\x1d]\x1dj\x1a!\x03h"e\x04ҟ +d>\x0c^{οy!\x0ev\n', '9\x1a)\x18\x0c:H\x000\x17\\Z\x15έ薿6f~YG"܇\x1cz\n', 'Bifh*\n', 'r\x14\x00SKBq\x1ch37\x000gB3ذ\x19lZ-0Q \x000N\x7f!肹\x06\x10wȗCe)@+i2i`\x1e\\h}\x14F&f\t L/|MuZ\x0f,\x00;_J\x18q=3ض4ߦ1K#SVS">R~y볟Ϊ}WL!CXե?\x19.Bc\x05\x08iq-\x197(zng4\x1b/ɿ\x18.֤\x02\x13nH\x14wPӜU3\x18&\x0c^mv\x020\x141EP~8^JK02F9>_y

In [3]:
# b = np.empty((0, 2), dtype = 'str')
# to_append = np.empty(0, dtype = 'str')
# to_append = np.append(to_append, ['hi'], axis = 0)
# to_append = np.append(to_append, ['hi'], axis = 0)
# print(to_append)
# print(to_append.shape)
# b = np.append(b, [to_append], axis = 0)
# print(b)

In [4]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [5]:
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [6]:
# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)


# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X


# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

In [7]:
# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

In [8]:
def read_data_diff(): 
    src_data  = open("giga-fren.release2.fixed.fr", "r")
    tgt_data = open("giga-fren.release2.fixed.en", "r")

#     dataset = np.ndarray((22521, 2), dtype='str')


    dataset = np.empty((0, 2), dtype = 'str')
    
    
    i = 0
    with open("giga-fren.release2.fixed.fr", "r") as src_data:
        with open("giga-fren.release2.fixed.en", "r") as tgt_data:
            for line1, line2 in zip(src_data, tgt_data):
                if i%100000==0: 
                    to_append = np.empty(0, dtype = 'str')
                    to_append = np.append(to_append, [line1], axis = 0)
                    to_append = np.append(to_append, [line2], axis = 0)
                    dataset = np.append(dataset, [to_append], axis = 0)
                if (i % 100000 == 0): print(i//100000)
                i+=1

    return dataset


In [17]:
dataset = read_data_diff()


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

In [28]:
# dataset = np.ndarray(dataset)
# print(dataset[:,1])

In [36]:
print(len(dataset))
dataset_ = dataset
print(len(dataset_))
train_ = dataset[:int(len(dataset)*0.9)]
test_ = dataset[int(len(dataset)*0.9):]
print(len(train_))

226
226
203


In [37]:
# load datasets
# dataset = load_clean_sentences('english-german-both.pkl')
# train = load_clean_sentences('english-german-train.pkl')
# test = load_clean_sentences('english-german-test.pkl')


# format of dataset: [[src, tgt], [src, tgt]...]
# print(len(dataset))
# print(len(train), len(test)) 
# print(dataset)

In [38]:
# print(train[:, 1])
# print(train[:, 0])
# print(dataset[0][0])
# print(dataset_[0])
# print(type(dataset))

In [39]:
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))

# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('German Vocabulary Size: %d' % ger_vocab_size)
print('German Max Length: %d' % (ger_length))


English Vocabulary Size: 2417
English Max Length: 678
German Vocabulary Size: 2137
German Max Length: 631


In [40]:
# prepare training data
# trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
# trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
# trainY = encode_output(trainY, eng_vocab_size)

trainX = encode_sequences(ger_tokenizer, ger_length, train_[:, 1])
print("hi")
trainY = encode_sequences(eng_tokenizer, eng_length, train_[:, 0])
print("hi")
trainY = encode_output(trainY, eng_vocab_size)
print("hi")


# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test_[:, 1])
print("hi")

testY = encode_sequences(eng_tokenizer, eng_length, test_[:, 0])
print("hi")

testY = encode_output(testY, eng_vocab_size)

print(trainX)

hi
hi
hi
hi
hi
[[ 428  786  428 ...    0    0    0]
 [  23    1  432 ...    0    0    0]
 [ 274   83  275 ...    0    0    0]
 ...
 [   1  165    2 ...    0    0    0]
 [   1  757  101 ...    0    0    0]
 [   1 1946   17 ...    0    0    0]]


In [41]:
# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
# Training Encoder-Decoder model to represent word embeddings and finally
# save the trained model as 'model.h5'

# summarize defined model
print(model.summary())
#plot_model(model, to_file='model.png', show_shapes=True)
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=30, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 631, 256)          547072    
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 678, 256)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 678, 256)          525312    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 678, 2417)         621169    
Total params: 2,218,865
Trainable params: 2,218,865
Non-trainable params: 0
_________________________________________________________________
None
Train on 203 samples, validate on 23 samples
Epoch 1/30
 - 50s - loss: 7.7681 - val_loss: 7.5315

Epoch 00001: val_loss improve

In [ ]:
from pickle import load
from numpy import array, argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()

def load_dataset(filename):
    return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines
):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# Map an integer to a word
def map_int_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# Predict the target sequence
def predict_sequence(model, tokenizer, source):
    pred = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in pred]
    target = list()
    for i in integers:
        word = map_int_to_word(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

# Evaluate the model
def evaluate_model(model, tokenizer, source, raw_dataset):
    predicted, actual = list(), list()
    for i, source in enumerate(source):
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, tokenizer, source)
        raw_target, raw_source = raw_dataset[i]
        if i < 10:
            print('src=[%s], target=[%s], predicted=[%s]' % (raw_source, raw_target, translation))
        actual.append(raw_target.split())
        predicted.append(translation.split())

    # Bleu Scores
    print('Bleu-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0),smoothing_function=chencherry.method4))
    print('Bleu-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0),smoothing_function=chencherry.method4))
    print('Bleu-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0),smoothing_function=chencherry.method4))
    print('Bleu-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=chencherry.method4))

# Load datasets
dataset = load_dataset('english-german-both.pkl')
train = load_dataset('english-german-train.pkl')
test = load_dataset('english-german-test.pkl')

# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])

# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])

# Prepare data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])

model = load_model('model.h5')

print('Testing on trained examples')
evaluate_model(model, eng_tokenizer, trainX, train)

print('Testing on test examples')
evaluate_model(model, eng_tokenizer, testX, test)
